In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
data = pd.read_excel("inventaire strat.xlsx")
data.head()

,Nom,Nb joueurs,Duree,Etat,Genre,Prompt,Jeux similaires
0,Splendor,2-4,moyen,NaN,"Stratégie, Gestion de ressources",Quel jeu de stratégie à 3 joueurs ?,"Azul, Majesty, Takenoko, Jaipur"
1,Princes de Florence,2-5,long,NaN,"Stratégie, Placement",Quel jeu de stratégie avec placement à 4 joueu...,"Caylus, 7 Wonders, Vinci, Civilisation"
2,Taluva,2-4,long,NaN,"Stratégie, Pose de tuiles",Quel jeu de stratégie avec pose de tuiles à 2 ...,"Carcassonne, Kingdomino, Kahuna"
3,Ave Caesar,3-6,moyen,NaN,Course,Quel jeu de course pour 5 joueurs ?,"Heat, Formula Dé, Scotland Yard"
4,Metroplys,2-4,long,NaN,"Stratégie, Pose de tuiles",Quel jeu de stratégie avec pose de tuiles à 3 ...,"Les Colons de Catan, Carcassonne, Risk"


In [134]:
data["Prompt"] = data["Prompt"].apply(lambda x: ''.join('_' if c.isdigit() else c for c in x))


In [17]:
import random
import numpy as np


def phi3_input(entry):
    instruction_text = (
        f"<|user|>\n{entry['instruction']}"
    )

    input_text = f"\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text


def create_prompt(data):
    prompts_list = []
    for index, row in data.iterrows():
        
        nb = row["Nb joueurs"]
        genre = row["Genre"]
        
        if pd.isna(nb) or pd.isna(genre):
            continue
        
        genre = genre.split(", ")
        
        if "-" in str(nb):
            min_val, max_val = map(int, nb.split("-"))
        elif "+" in str(nb):
            min_val = int(nb.split("+")[0])
            max_val = min_val + 10
        else:
            min_val = max_val = int(nb)
        possible_nb = list(np.arange(min_val, max_val + 1))
        possible_genre = genre[:]
        
        prompts = []        
        tirage_nb = random.choice(possible_nb)
        tirage_genre = random.choice(possible_genre)

        instruction = f"Quel jeu {row['Duree']} de {tirage_genre} pour {tirage_nb} joueurs ?"
        output = row['Nom']
        inputs = {"joueurs": [min_val, max_val] if max_val>min_val else min_val, "genre": genre, "duree": row["Duree"]}
        prompt = {
            "instruction": instruction,
            "input": inputs,
            "output": output
        }
        prompts_list.append(prompt)
    return prompts_list
        

In [18]:
prompts = create_prompt(data)
prompts[:5]

[{'instruction': 'Quel jeu moyen de Gestion de ressources pour 3 joueurs ?',
  'input': {'joueurs': [2, 4],
   'genre': ['Stratégie', 'Gestion de ressources'],
   'duree': 'moyen'},
  'output': 'Splendor'},
 {'instruction': 'Quel jeu long de Placement pour 3 joueurs ?',
  'input': {'joueurs': [2, 5],
   'genre': ['Stratégie', 'Placement'],
   'duree': 'long'},
  'output': 'Princes de Florence'},
 {'instruction': 'Quel jeu long de Pose de tuiles pour 4 joueurs ?',
  'input': {'joueurs': [2, 4],
   'genre': ['Stratégie', 'Pose de tuiles'],
   'duree': 'long'},
  'output': 'Taluva'},
 {'instruction': 'Quel jeu moyen de Course pour 5 joueurs ?',
  'input': {'joueurs': [3, 6], 'genre': ['Course'], 'duree': 'moyen'},
  'output': 'Ave Caesar'},
 {'instruction': 'Quel jeu long de Stratégie pour 3 joueurs ?',
  'input': {'joueurs': [2, 4],
   'genre': ['Stratégie', 'Pose de tuiles'],
   'duree': 'long'},
  'output': 'Metroplys'}]

In [19]:
import json
def create_instruction_file(data):
    prompts = create_prompt(data)
    with open("instruction.json", "w", encoding="utf-8") as f:
        json.dump(prompts, f, ensure_ascii=False, indent=4)

create_instruction_file(data)

        